<div class="alert alert-block alert-warning">

# Aquisition and Prep Exercises

<div class="alert alert-block alert-success">

#### Exercises I
Let's review the steps we take at the beginning of each new module.

1. Create a new repository named regression-exercises in your GitHub; all of your Regression work will be housed here.
2. Clone this repository within your local codeup-data-science directory.
3. Create a .gitignore and make sure your list of 'files to ignore' includes your env.py file.
4. Ceate a README.md file that outlines the contents and purpose of your repository.
5. Add, commit, and push these two files.
6. Now you can add your env.py file to this repository to access the Codeup database server.
7. For these exercises, you will create wrangle.ipynb and wrangle.py files to hold necessary functions.
8. As always, add, commit, and push your work often.

<div class="alert alert-block alert-success">

#### Exercises II
Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from env import get_connection
import os

<div class="alert alert-block alert-info">

1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [53]:
#

# table namne
url = "zillow"

# the query.  261 = single family home    
sql_query = '''
            SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips 
            FROM properties_2017 
            WHERE propertylandusetypeid = 261
            '''

# create df from the query table
df = pd.read_sql(sql_query, get_connection(url))

In [54]:
#check df
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


<div class="alert alert-block alert-info">

2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [55]:
# Check for nulls
df.isna().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 115.0 MB


In [40]:
filtered_df = df[(np.isnan(df['calculatedfinishedsquarefeet'])) & (df['bedroomcnt'] == 0)]

In [41]:
filtered_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
5,0.0,0.0,NaN,124.0,NaN,NaN,6037.0


In [42]:
# Drop nulls. Most of them had a combination of no BR, Bath and SQF infomation. 
df = df.dropna()

In [43]:
#Rename the columns
df = df.rename(columns = {'bedroomcnt':'bedrooms', 'bathroomcnt':'bathrooms', 'calculatedfinishedsquarefeet':'area', 'taxvaluedollarcnt':'tax_value', 'yearbuilt':'year_built'})

In [44]:
df.head()

,bedrooms,bathrooms,area,tax_value,year_built,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [45]:
# First lets get the names of all the columsn
df.columns

Index(['bedrooms', 'bathrooms', 'area', 'tax_value', 'year_built', 'taxamount',
       'fips'],
      dtype='object')

In [46]:
# print a list of unique values for each column
column_list = ['bedrooms', 'bathrooms','area']
for column in column_list:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print()


Unique values in column 'bedrooms':
[ 4.  3.  0.  5.  2.  1.  6.  7.  8.  9. 25. 10. 11. 14. 15. 13. 12. 16.
 18.]

Unique values in column 'bathrooms':
[ 2.    4.    0.    1.    2.5   3.5   3.    5.5   4.5   5.    1.5   7.5
  6.   10.    8.    7.   11.    9.    8.5   6.5  13.    9.5  20.   19.5
 12.   10.5  15.   16.    1.75 14.   18.    0.5  17.   12.5  11.5  14.5
 32.   19.  ]

Unique values in column 'area':
[ 3633.  1620.  2077. ...  8545.  7342. 24920.]



In [47]:
#test columns to see if there are decimal points. 
filtered_df = df[df['area'] % 1 != 0]

In [48]:
filtered_df.head()

,bedrooms,bathrooms,area,tax_value,year_built,taxamount,fips


In [49]:
# change types, keeping bathrooms and taxamount as float. 
df.bedrooms = df.bedrooms.astype('int')
df.area = df.area.astype('int')
df.tax_value = df.tax_value.astype('int')
df.year_built = df.year_built.astype('int')
df.fips = df.fips.astype('int')

In [50]:
df.head()

,bedrooms,bathrooms,area,tax_value,year_built,taxamount,fips
4,4,2.0,3633,296425,2005,6941.39,6037
6,3,4.0,1620,847770,2011,10244.94,6037
7,3,2.0,2077,646760,1926,7924.68,6037
11,0,0.0,1200,5328,1972,91.60,6037
14,0,0.0,171,6920,1973,255.17,6037


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   bedrooms    int64  
 1   bathrooms   float64
 2   area        int64  
 3   tax_value   int64  
 4   year_built  int64  
 5   taxamount   float64
 6   fips        int64  
dtypes: float64(2), int64(5)
memory usage: 130.6 MB


In [52]:
# 5. Save cleaned data to csv
df.to_csv('zillow_data.csv',index=False)

<div class="alert alert-block alert-info">

3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [57]:
def acquire_zillow():

    # table namne
    url = "zillow"

    # the query.  261 = single family home    
    sql_query = '''
                SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips 
                FROM properties_2017 
                WHERE propertylandusetypeid = 261
                '''

    # create df from the query table
    df = pd.read_sql(sql_query, get_connection(url))

    return df

In [58]:
def wrangle_zillow(df):
    
    # Drop nulls
    df = df.dropna()
    
    # Rename the columns
    df = df.rename(columns = {'bedroomcnt':'bedrooms', 'bathroomcnt':'bathrooms', 'calculatedfinishedsquarefeet':'area', 'taxvaluedollarcnt':'tax_value', 'yearbuilt':'year_built'})


    # Convert data types
    df.bedrooms = df.bedrooms.astype('int')
    df.area = df.area.astype('int')
    df.tax_value = df.tax_value.astype('int')
    df.year_built = df.year_built.astype('int')
    df.fips = df.fips.astype('int')

    # Save to csv
    df.to_csv('zillow_data.csv',index=False)

    return df 
    